In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import copy as cp

def initialize_lattice(N):
    lattice = np.random.choice([1, -1], size=N)
    return lattice

def get_neighbors(lattice, i):
    N = lattice.shape[0]
    neighbors = []
    neighbors = [(i+1)%N, (i-1)%N]
    return neighbors


# Perform one Wolff step (with cluster flip)
def flip_cluster(lattice, cluster):
    # Flip all spins in the cluster
    for i in cluster:
        lattice[i] = -lattice[i]

def wolff_step(lattice, J, T):
    # Perform one Wolff step (no cluster flip)
    N = lattice.shape[0]
    i = np.random.randint(N)
    s = lattice[i]
    stack = [i]
    cluster = set([i])
    while len(stack) > 0:
        i = stack.pop()
        neighbors = get_neighbors(lattice, i)
        for n in neighbors:
            if lattice[n] == s and n not in cluster and np.random.rand() < 1 - np.exp(-2 *J / T):
                stack.append(n)
                cluster.add(n)
    flip_cluster(lattice, cluster)
    m = abs(np.sum(lattice)) / N    
    if m > 0.7:
        magnetization = abs(np.sum(lattice))
    else:
        magnetization = np.sum(lattice)
    return lattice, magnetization

# # Set the parameters for the simulation
# def calculate_energy(J,lattice):
#     # Calculate the energy of a lattice
#     N = lattice.shape[0]
#     energy = 0
#     for i in range(N):
#         for j in range(N):
#             s = lattice[i]
#             neighbors = get_neighbors(lattice, i)
#             for n in neighbors:
#                 energy += J* s * lattice[n]
#     return energy / 2  # Each interaction is counted twice

def simulate_wolff(N, J, T, num_steps):
    # Simulate the Ising model using the Wolff algorithm
    lattice = initialize_lattice(N)
    magnetizations = []
    states = []
    for step in range(num_steps):
        lattice, magnetization = wolff_step(lattice, J, T)
        magnetizations.append(magnetization)
        states.append(cp.deepcopy(lattice))
    #plt.imshow(states_array, interpolation='nearest', cmap=plt.cm.binary)
    # # states = np.transpose(states)
    # states_array = np.array(states)
    # cmap = plt.cm.binary
    # plt.imshow(states_array, cmap=cmap, aspect='auto')
    # plt.xlabel('Lattice Point')
    # plt.ylabel('Time Step')
    # plt.title('Visualization of Sts')
    # plt.show() 

    return lattice, magnetizations, states

# Set the parameters for the simulation
N_lst = [300, 400, 500, 600]    # Size of the lattice
T_lst = np.linspace(0.001,3,50)  # Temperature
J = 0.5  # coorelation parameter
num_steps = 50000  # Number of simulation steps
magnetizations_lst = []
Mag_real_flu = []
largest_value_lst = np.zeros((len(T_lst), len(N_lst))) 
second_largest_value_lst = np.zeros((len(T_lst), len(N_lst))) 
third_largest_value_lst = np.zeros((len(T_lst), len(N_lst))) 
for j, N in enumerate(N_lst):
    for i, T in enumerate(T_lst):
        # Run the simulation
        lattice, magnetizations, states = simulate_wolff(N, J, T, num_steps)

        # Initialize an empty list to store the selected states
        steps = []

        # Select states every 205 Monte Carlo steps
        for k in range(200, num_steps, 205):
            steps.append(states[k])

        # transport the selected states
        transposed_steps = np.transpose(steps)
        U, S, VT = np.linalg.svd(transposed_steps)

        # renormalize the S
        sqrt_sum_sqare_S = np.sqrt(np.sum(np.square(S)))
        S /= sqrt_sum_sqare_S

        # sort S from max to min
        sorted_S = sorted(S, reverse=True)
        largest_value = sorted_S[0]
        second_largest_value = sorted_S[1]
        third_largest_value = sorted_S[2]


        # Calculate the mean and real_fluctuation of magnetizations from step 100 to the end
        mean_mag = np.mean(magnetizations[200:])
        variance_mag = np.var(magnetizations[200:])
        rel_fluctuation_mag = np.sqrt(variance_mag) / mean_mag
        mean_mag_individual = mean_mag / (N * N)
        magnetizations_lst.append(mean_mag_individual)
        largest_value_lst[i, j] = largest_value
        second_largest_value_lst[i, j] = second_largest_value
        third_largest_value_lst[i, j] = third_largest_value


    # if the real_fluctuation_mag over 0.1, let magnezation be zero
 
for j, N in enumerate(N_lst):
    plt.plot(T_lst, largest_value_lst[:, j], label=f'N={N}')
plt.xlabel('Temperature')
plt.ylabel('Largest Value')
plt.legend()
plt.show()

# # Print mean and real_fluctuation of magnetizations
# print(f"Mean of magnetizations from step 200 to the end: {mean}")
# print(f'Relative fluctuation of magnetization: {rel_fluctuation:.4f}')
# print(S)

# Plot the energy and megnetization vs. step
# fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
# ax1.plot(energies)
# ax1.set_ylabel('Energy')
# ax2.plot(magnetizations)
# ax2.set_xlabel('Step')
# ax2.set_ylabel('Magnetization')
# plt.show()